In [ ]:
# Calculate AUROC and AUPR for Chymera screen scored using 6 scoring methods.

In [ ]:
# Uncomment the command below if you run this script separately. 
#%run AnalysisFunctions.ipynb


In [ ]:
#filtered = ""

In [ ]:
print("Running Chymera Analysis")


print(f"Apply Filter?: {apply_filter}")

if apply_filter.lower() == "true":
    filtered = "/Filtered"
    print("Compiling results with filtered pairs")
else:
    filtered = ""
    print("Compiling results with unfiltered pairs")

In [ ]:
# Load the zdLFC scores
# zdLFC does not filter out anything so filtered = unfiltered 
zdLFC_HAP1 = pd.read_csv("../zdLFC Scripts/zdLFC Output/ChymeraHAP1.csv")
zdLFC1_RPE1 = pd.read_csv("../zdLFC Scripts/zdLFC Output/ChymeraRPE1.csv")

zdLFC_HAP1 = zdLFC_HAP1.rename(columns={zdLFC_HAP1.columns[0]: 'GENE'})
zdLFC_HAP1 = zdLFC_HAP1.set_index('GENE')

zdLFC1_RPE1 = zdLFC1_RPE1.rename(columns={zdLFC1_RPE1.columns[0]: 'GENE'})
zdLFC1_RPE1 = zdLFC1_RPE1.set_index('GENE')

zdLFC = pd.concat([zdLFC_HAP1, zdLFC1_RPE1], axis=1)
zdLFC = reindex_alphbetically(zdLFC) # Make sure they are alphabetically ordered

zdLFC = -1 * zdLFC # less score -> SL 
zdLFC

In [ ]:
orthrus = pd.read_csv("../Orthrus Scripts/OrthrusOutput" + filtered + "/chymera_orthrus.tsv", sep='\t')
orthrus['GENE'] = orthrus['gene1'] + "_" + orthrus['gene2']
orthrus = orthrus.filter(like='differential', axis=1) \
                    .join(orthrus.filter(items=['GENE']))

orthrus = orthrus.rename(columns={'differential_combn_vs_single_HAP1_T12': 'HAP_T12', 'differential_combn_vs_single_HAP1_T18': 'HAP_T18', 'differential_combn_vs_single_RPE1_T18': 'RPE_T18',
                                  'differential_combn_vs_single_RPE1_T24': 'RPE_T24'})
orthrus = orthrus.set_index('GENE')

orthrus = reindex_alphbetically(orthrus)
orthrus = orthrus * -1


In [ ]:
gemini_strong_HAP1 = pd.read_csv("../Gemini Scripts/GeminiOutput" + filtered + "/Gemini_Chymera_HAP1_Strong.csv")
gemini_strong_HAP1 = gemini_strong_HAP1.rename(columns={gemini_strong_HAP1.columns[0]: 'gene_pair'})
gemini_strong_HAP1['gene_pair'] = gemini_strong_HAP1['gene_pair'].str.replace(';', '_')
gemini_strong_HAP1 = gemini_strong_HAP1.set_index('gene_pair')
gemini_strong_HAP1 = reindex_alphbetically(gemini_strong_HAP1)
gemini_strong_HAP1

gemini_strong_RPE1 = pd.read_csv("../Gemini Scripts/GeminiOutput" + filtered + "/Gemini_Chymera_RPE1_Strong.csv")
gemini_strong_RPE1 = gemini_strong_RPE1.rename(columns={gemini_strong_RPE1.columns[0]: 'gene_pair'})
gemini_strong_RPE1['gene_pair'] = gemini_strong_RPE1['gene_pair'].str.replace(';', '_')
gemini_strong_RPE1 = gemini_strong_RPE1.set_index('gene_pair')
gemini_strong_RPE1 = reindex_alphbetically(gemini_strong_RPE1)


In [ ]:
gemini_SensL_HAP1 = pd.read_csv("../Gemini Scripts/GeminiOutput" + filtered + "/Gemini_Chymera_HAP1_sensitive_lethality.csv")
gemini_SensL_HAP1 = gemini_SensL_HAP1.rename(columns={gemini_SensL_HAP1.columns[0]: 'gene_pair'})
gemini_SensL_HAP1['gene_pair'] = gemini_SensL_HAP1['gene_pair'].str.replace(';', '_')
gemini_SensL_HAP1 = gemini_SensL_HAP1.set_index('gene_pair')
gemini_SensL_HAP1 = reindex_alphbetically(gemini_SensL_HAP1)
gemini_SensL_HAP1


gemini_SensL_RPE1 = pd.read_csv("../Gemini Scripts/GeminiOutput" + filtered + "/Gemini_Chymera_RPE1_sensitive_lethality.csv")
gemini_SensL_RPE1 = gemini_SensL_RPE1.rename(columns={gemini_SensL_RPE1.columns[0]: 'gene_pair'})
gemini_SensL_RPE1['gene_pair'] = gemini_SensL_RPE1['gene_pair'].str.replace(';', '_')
gemini_SensL_RPE1 = gemini_SensL_RPE1.set_index('gene_pair')
gemini_SensL_RPE1 = reindex_alphbetically(gemini_SensL_RPE1)


In [ ]:
# No pairs dropped in Parrish
parrish_score = pd.read_csv('../Parrish Score Scripts/ParrishOutput/Chymera_Parrish.csv')
parrish_score.set_index('paralog_pair', inplace=True)
parrish_score = reindex_alphbetically(parrish_score)
parrish_score = parrish_score * -1

parrish_score

In [ ]:
# These figures are not included in the paper. 
Chymera_HAP1_T18 = pd.concat([ # remove T12, its ETP, not needed as per slkb
    zdLFC['HAP1.T18'], 
    orthrus['HAP_T18'], 
    gemini_strong_HAP1['HAP1.T18'],
    gemini_SensL_HAP1['HAP1.T18'],
    
    parrish_score['GI_score_HAP1']
    
], axis = 1, keys= scores)

#Chymera_HAP1_T12
results_Chymera_HAP1_LTP =Cell_Line_Analysis(Chymera_HAP1_T18, 'HAP1.T18', 'CHyMErA', scores, ground_truth_depmap_hit)



In [ ]:

Chymera_RPE1_T24 = pd.concat([zdLFC['RPE1.T24'], orthrus['RPE_T24'],
                             gemini_strong_RPE1['RPE1.T24'],
                             gemini_SensL_RPE1['RPE1.T24'], 
                             
                              parrish_score['GI_score_RPE1']], axis = 1, keys=scores)

results_Chymera_RPE1_LTP = Cell_Line_Analysis(Chymera_RPE1_T24, 'RPE1.T24', 'CHyMErA', scores, ground_truth_depmap_hit)


In [ ]:

full_list = [results_Chymera_HAP1_LTP, results_Chymera_RPE1_LTP]

roc, pr = ConvertResultsToDF(full_list )

## Write results to a file
roc.to_csv("Output" + filtered + "/Chymera_ROC_DepMapHits.csv", index=False)
pr.to_csv("Output" + filtered + "/Chymera_PR_DepMapHits.csv", index=False)


In [ ]:

# combine all cell lines and then repeat analysis
chymera_all = pd.concat([Chymera_HAP1_T18, Chymera_RPE1_T24])
results_chymera_all = Cell_Line_Analysis(chymera_all, 'All', 'CHyMErA', scores, ground_truth_depmap_hit)

roc_all, pr_all = ConvertResultsToDF([results_chymera_all] )
## Write results to a file
roc_all.to_csv("Output" + filtered+ "/Chymera_All_ROC_DepMapHits.csv", index=False)
pr_all.to_csv("Output" + filtered + "/Chymera_All_PR_DepMapHits.csv", index=False)


In [ ]:
 ##### NOW USE GROUND TRUTH KORLEFE BENCHMARK  #######
chymera_all = pd.concat([Chymera_HAP1_T18, Chymera_RPE1_T24])

results_Chymera_HAP1_LTP_korlefe =Cell_Line_Analysis(Chymera_HAP1_T18, 'HAP1.T18', 'CHyMErA', scores, Koferle)
results_Chymera_RPE1_LTP_korlefe =Cell_Line_Analysis(Chymera_RPE1_T24, 'RPE1.T24', 'CHyMErA', scores, Koferle)

full_list_Koferle = [results_Chymera_HAP1_LTP_korlefe, results_Chymera_RPE1_LTP_korlefe]
roc_Koferle, pr_Koferle = ConvertResultsToDF(full_list_Koferle )


roc_Koferle.to_csv("Output" + filtered+ "/Chymera_ROC_Koferle.csv", index=False)
pr_Koferle.to_csv("Output" + filtered+ "/Chymera_PR_Koferle.csv", index=False)


results_chymera_all_Koferle = Cell_Line_Analysis(chymera_all, 'All', 'CHyMErA', scores, Koferle)

roc_all_Koferle, pr_all_Koferle = ConvertResultsToDF([results_chymera_all_Koferle] )
## Write results to a file

roc_all_Koferle.to_csv("Output" + filtered + "/Chymera_All_ROC_Koferle.csv", index=False)
pr_all_Koferle.to_csv("Output" + filtered + "/Chymera_All_PR_Koferle.csv", index=False)



In [ ]:
#chymera_all = pd.concat([Chymera_HAP1_T18, Chymera_RPE1_T24])
output_path = "Output" + filtered + "\Comparison"
# thompson_all = pd.concat([Thompson_A375, Thompson_MEWO, Thompson_RPE])
chymera_all.corr(method = "spearman").to_csv(output_path + "\Chymera_Correlations.csv")

In [ ]:
Chymera_HAP1_T18_J = jaccard_similarity(Chymera_HAP1_T18, 0.95)
Chymera_HAP1_T18_J.to_csv(output_path + "\Chymera_HAP1_Jaccard.csv")

Chymera_RPE1_T24_J = jaccard_similarity(Chymera_RPE1_T24, 0.95)
Chymera_RPE1_T24_J.to_csv(output_path + "\Chymera_RPE1_Jaccard.csv")

Chymera_Jaccard = (Chymera_HAP1_T18_J + Chymera_RPE1_T24_J) / 2
Chymera_Jaccard

In [ ]:
Chymera_Jaccard.to_csv(output_path + "\Chymera_Jaccard.csv")